# CS202: Compiler Construction

## In-class Exercises, Week of 04/18/2022

----

# Part 1: Functions and Lfun

## Question 1

Write an `Rfun` program with a function called `add1` that adds 1 to its argument, and a call to the function.

def add1(n: int) -> int:
    return n + 1

print(add1(100))

## Question 2

Write a recursive program to calculate the factorial of 5.

def fact(n: int) -> int:
    return fact(n - 1) + fact(n - 2)

----

# Part 2: Typechecking for Functions

## Question 3

What is the type of the function `add1`?

Callable[[int], int]

## Question 4

What is the type of the function `*`

Callable[[int, int], int]

## Question 5

Why do we need to specify the types of a function's arguments?

when we have ambiguity we need to be able to make a call on what type something is, and we also need to be able to typecheck recursive function calls.

## Question 6

Write a function `length` such that the following expression returns 3:

```
v = (1, (2, (3, 0)))
print(length(v))
```

def length(tup: List[int]) -> int:
    if tup[1] == 0:
        return 1
    else:
        return 1 + length(tup[1])
# you cannot write this in Lfun, our types are not expressive enough.

## Question 7

How do we typecheck a function call of the form `f(a1, ..., ak)`

check that f has k parameters
check that each parameter ti has the type of ai

## Question 8

How do we typecheck a function definition?

We create a new environment from the outer scope with the parameters and their types, then check the statements to make sure the parameter types, any local variable types, and the return type is consistent. Then we return back to the global environment.

## Question 9

How do we typecheck a `Lfun` program?

First we find all the functions and their types and add them to the top-level environment
Then we typecheck all the function definitions with the knowledge of the other functions
Then we typecheck all the top-level statements

----

# Part 3: Functions in x86 Assembly

## Question 10

Write an x86 assembly program corresponding to the `add1` program.

In [1]:
from cs202_support.eval_x86 import X86Emulator

asm = """
add1:
    movq %rdi, %r8
    addq $1, %r8
    movq %r8, %rax
    retq
main:
    movq $5, %rdi
    callq add1
"""

emu = X86Emulator(logging=True)
emu.eval_program(asm)
emu.print_state()

============================== STARTING EXECUTION ==============================
FINAL STATE:
  Location                        Value
0  reg rbp                         1000
1  reg rsp                         1000
2  reg rdi                            5
3   reg r8                            6
4  reg rax                            6
5     add1  FunPointer(fun_name='add1')
6     main  FunPointer(fun_name='main')
OUTPUT: []
============================== FINISHED EXECUTION ==============================


,Location,Value
0,reg rbp,1000
1,reg rsp,1000
2,reg rdi,5
3,reg r8,6
4,reg rax,6
5,add1,FunPointer(fun_name='add1')
6,main,FunPointer(fun_name='main')


## Question 11

Describe the *calling convention* we will use for functions in Rfun

Arguments go in registers `rdi, rsi, rdx, rcx, r8, r9` in that order
If a function has >6 arguments, put arguments 6-n in a tuple and put that in r9
Return values go in `rax`
Functions should handle their own stack frames, and jump back to the right spot


## Question 12

Describe the management of the *stack* and *root stack* performed on function entry and exit.

on entry:
Allocate new stack frame with enough slots
Allocate new root stack frame with enough slots
on exit:
reclaim the two stack frames
retq

## Question 13

Modify the program from earlier to correctly manage the stack and root stack. Allocate the variable `n` on the stack.

In [ ]:
asm = """
add1_main:
    pushq %rbp
    movq %rsp, %rbp
    subq $16, %rsp
    jmp add1_start

add1_start:
    movq %rdi, -8(%rbp)
    addq $1, -8(%rbp)
    movq -8(%rbp), %rax
    jmp add1_conclusion

add1_conclusion:
    addq $16, %rsp
    popq %rbp
    retq
"""

emu = X86Emulator(logging=True)
emu.eval_program(asm)
emu.print_state()

## Question 14

Modify the program again, to save and restore the *callee-saved registers*.

In [ ]:
asm = """
add1_main:
    pushq %rbp
    movq %rsp, %rbp
    pushq %rbx
    pushq %r12
    pushq %r13
    pushq %r14
    subq $16, %rsp
    jmp add1_start

add1_start:
    movq %rdi, -8(%rbp)
    addq $1, -8(%rbp)
    movq -8(%rbp), %rax
    jmp add1_conclusion

add1_conclusion:
    addq $16, %rsp
    pushq %r14
    pushq %r13
    pushq %r12
    pushq %rbx
    popq %rbp
    retq
"""

emu = X86Emulator(logging=True)
emu.eval_program(asm)
emu.print_state()